In [1]:
import pandas as pd
import holoviews as hv
import hvplot.pandas

from bokeh.models import DatetimeTickFormatter
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

import colorcet as cc

hv.extension('bokeh')


%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
json_file = '/home/jeremie/monitor/s1_2023_timeseries_ant.json'  # Replace with your JSON file path
df = pd.read_json(json_file)
# 2. Convert the date columns to datetime objects
df['date1'] = pd.to_datetime(df['date1'], format='%y-%m-%d')
df['date2'] = pd.to_datetime(df['date2'], format='%y-%m-%d')

# We'll "melt" the dataframe so that both date columns are in a single column, making it easier to plot
melted_df = df.melt(id_vars=['pairs'], value_vars=['date1', 'date2'], value_name='date').drop(columns='variable')
# Group by date and get the count
date_counts = melted_df.groupby('date').size().reset_index(name='count')

In [3]:
# Use the blue color palette from colorcet and get 12 uniformly spread values
blue_palette = cc.glasbey_cool[::-1]
# Create a dictionary that maps each month to a shade of blue
month_to_color = {month: blue_palette[(month-1) * len(blue_palette) // 12] for month in range(1, 13)}
# Assign colors based on the month
date_counts['color'] = date_counts['date'].dt.month.map(month_to_color)


# Define a custom hook to modify the underlying Bokeh plot's x-axis ticks using DatetimeTickFormatter
def adjust_ticks(plot, element):
    plot.handles['xaxis'].formatter = DatetimeTickFormatter(
        days="%d  %m  %Y",
        months="%d  %m  %Y",
        years="%d %m %Y"
    )
    
bar_plot = date_counts.hvplot.bar(x='date', y='count', width=1200, height=350, rot=90, line_alpha=0,color='color').opts(hooks=[adjust_ticks])
bar_plot

:Bars   [date]   (count,color)

In [4]:

# Bokeh output to Jupyter Notebook
output_notebook()

# Create bar plot
p = figure(x_axis_type="datetime", title="Images usage frequencies",height=350,width=1000)
p.vbar(x=date_counts["date"], top=date_counts["count"])

# Customize x-axis tick display
# Customize x-axis tick display
p.xaxis.formatter = DatetimeTickFormatter(
    days="%d %m %Y",
    months="%d %m %Y",
    years="%d %m %Y",
)
# This will ensure only one date is displayed over 5 days
p.xaxis.ticker.desired_num_ticks = len(date_counts["date"]) // 18

show(p)

Loading BokehJS ...

In [5]:
from colorcet.plotting import swatch, swatches
swatch('cyclic')

KeyError: 'cyclic'